[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arihantbansal/reddit-tagging/blob/main/flairScience.ipynb)

In [1]:
pip install transformers

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.preprocessing import LabelEncoder as LE
from transformers import DistilBertTokenizer, DistilBertModel

In [3]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
df = pd.read_csv("/content/raskscience_posts.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (70,72) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_template_id,link_flair_text,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,pwls,removed_by_category,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,title,total_awards_received,url,whitelist_status,wls,post_hint,preview,treatment_tags,upvote_ratio,author_cakeday,author_flair_background_color,author_flair_text_color,banned_by,edited,author_is_blocked,is_created_from_ads_ui,author_flair_template_id,steward_reports,distinguished,gilded
0,[],False,rockyzg,NaN,[],NaN,text,t2_wn46a,False,False,[],False,False,1584132787,self.askscience,https://www.reddit.com/r/askscience/comments/f...,{},fi6mq5,False,False,False,False,False,True,False,#aaddaa,med,[],78037cd8-1e22-11e3-b776-12313d096169,Medicine,dark,text,False,False,True,0,0,False,all_ads,/r/askscience/comments/fi6mq5/what_are_the_odd...,False,6,reddit,1584132789,1,[removed],True,False,False,askscience,t5_2qm4e,18755458,public,self,What are the odds of COVID-19 mutating into mo...,0,https://www.reddit.com/r/askscience/comments/f...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Mrs_Mousy,NaN,[],NaN,text,t2_3uujxz82,False,False,[],False,False,1584132452,self.askscience,https://www.reddit.com/r/askscience/comments/f...,{},fi6jdl,False,False,False,False,False,True,False,NaN,NaN,[],NaN,NaN,dark,text,False,False,True,2,0,False,all_ads,/r/askscience/comments/fi6jdl/is_caffeine_depe...,False,6,reddit,1584132459,1,[removed],True,False,False,askscience,t5_2qm4e,18755436,public,self,Is caffeine dependence detrimental to one's he...,0,https://www.reddit.com/r/askscience/comments/f...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,mageofsoap,NaN,[],NaN,text,t2_3ddv4y9x,False,False,[],False,False,1584132343,self.askscience,https://www.reddit.com/r/askscience/comments/f...,{},fi6ice,False,False,False,False,False,True,False,#aaddaa,med,[],327dd6a4-dfa6-11e3-adbc-12313d1a208b,Human Body,dark,text,False,False,True,0,0,False,all_ads,/r/askscience/comments/fi6ice/do_different_cou...,False,6,reddit,1584132349,1,[removed],True,False,False,askscience,t5_2qm4e,18755430,public,self,Do different country's accents have anything t...,0,https://www.reddit.com/r/askscience/comments/f...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,EmersonikMusic,NaN,[],NaN,text,t2_8onqy,False,False,[],False,False,1584132010,self.askscience,https://www.reddit.com/r/askscience/comments/f...,{},fi6f83,False,False,False,False,False,True,False,#99ffcc,bio,[],431d9b62-8971-11e1-abc8-12313d18ad57,Biology,dark,text,False,False,True,0,0,False,all_ads,/r/askscience/comments/fi6f83/lets_say_coronav...,False,6,reddit,1584132018,1,[removed],True,False,False,askscience,t5_2qm4e,18755414,public,self,Let’s say coronavirus or any virus WAS man-mad...,0,https://www.reddit.com/r/askscience/comments/f...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,just_a_corgi,NaN,[],NaN,text,t2_3f8biemm,False,False,[],False,False,1584131751,self.askscience,https://www.reddit.com/r/askscience/comments/f...,{},fi6csa,False,False,False,False,False,True,False,#cc99ff,physics,[],e8738d5c-8970-11e1-9266-12313d2c1af1,Physics,dark,text,False,False,True,21,0,False,all_ads,/r/askscience/comments/fi6csa/if_you_put_a_vac...,False,6,reddit,1584131758,1,[removed],True,False,False,askscience,t5_2qm4e,18755401,public,self,If you put a vacuum c

In [6]:
list(df.columns)

['all_awardings',
 'allow_live_comments',
 'author',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'domain',
 'full_link',
 'gildings',
 'id',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 'pwls',
 'removed_by_category',
 'retrieved_on',
 'score',
 'selftext',
 'send_replies',
 'spoiler',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_subscribers',
 'subreddit_type',
 'thumbnail',
 'title',
 'total_awards_r

In [7]:
df_flair = df[['title','link_flair_text']]
df_flair = df_flair.dropna(axis=0)
df_flair = df_flair.head(10000)

In [8]:
df_flair.head()

,title,link_flair_text
0,What are the odds of COVID-19 mutating into mo...,Medicine
2,Do different country's accents have anything t...,Human Body
3,Let’s say coronavirus or any virus WAS man-mad...,Biology
4,If you put a vacuum chamber in space can you c...,Physics
5,Why is eating fast for humans bad but not for ...,Biology


In [9]:
df_flair['link_flair_text'].unique()

array(['Medicine', 'Human Body', 'Biology', 'Physics', 'Earth Sciences',
       'Neuroscience', 'Astronomy', 'Paleontology', 'Engineering',
       'Chemistry', 'Mathematics', 'Economics', 'Psychology', 'Computing',
       'Planetary Sci.', 'Archaeology', 'COVID-19', 'Anthropology',
       'Social Science', 'Political Science', 'Linguistics'], dtype=object)

In [10]:
label=LE()
df_flair['link_flair_text']=label.fit_transform(df_flair['link_flair_text'])

In [11]:
df_flair.head()

,title,link_flair_text
0,What are the odds of COVID-19 mutating into mo...,13
2,Do different country's accents have anything t...,10
3,Let’s say coronavirus or any virus WAS man-mad...,3
4,If you put a vacuum chamber in space can you c...,16
5,Why is eating fast for humans bad but not for ...,3


In [12]:
titles = list(df_flair['title'].astype('str'))
labels = list(df_flair['link_flair_text'])

In [13]:
lengths = df["title"].str.len()
MAX_LEN = (max(lengths))

In [14]:
df_flair = df_flair.rename({'title': 'text', 'link_flair_text': 'labels'}, axis='columns')

In [15]:
class flair_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.text[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.labels[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [16]:
train_size = 0.70
train_dataset=df_flair.sample(frac=train_size,random_state=200)
test_valid_dataset=df_flair.drop(train_dataset.index)
test_size = 1/3
test_dataset=test_valid_dataset.sample(frac=test_size,random_state=200)
valid_dataset=test_valid_dataset.drop(test_dataset.index)

train_dataset=train_dataset.reset_index(drop=True)
valid_dataset=valid_dataset.reset_index(drop=True)
test_dataset=test_dataset.reset_index(drop=True)

In [17]:
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VAL Dataset: {}".format(valid_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

TRAIN Dataset: (7000, 2)
VAL Dataset: (2000, 2)
TEST Dataset: (1000, 2)


In [18]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [19]:
train_set = flair_Dataset(train_dataset, tokenizer, MAX_LEN)
val_set = flair_Dataset(valid_dataset, tokenizer, MAX_LEN)
test_set = flair_Dataset(test_dataset, tokenizer, MAX_LEN)

In [20]:
train_set[0]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'ids': tensor([  101,  2009,  1674,  3538,  5080,  6047,  1193,  1472,  1170,  1217,
          2023,  1107,   170, 10410,  3201, 15761,  1111,  1256,   170, 10298,
          2971,  1104,  1159,   136,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [21]:
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 0
         }

train_dl = DataLoader(train_set, **params)
valid_dl = DataLoader(val_set, **params)
test_dl = DataLoader(test_set, **params)

In [22]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 21)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [40]:
model = DistillBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [41]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-04)

In [42]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [43]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(train_dl, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [44]:
def valid(model, valid_dl):
    model.eval()
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(valid_dl, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [45]:
for epoch in range(6):
    train(epoch)
    acc = valid(model, valid_dl)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 5000 steps: 3.0037317276000977
Training Accuracy per 5000 steps: 12.5
The Total Accuracy for Epoch 0: 21.87142857142857
Training Loss Epoch: 2.4671396833576567
Training Accuracy Epoch: 21.87142857142857
Validation Loss Epoch: 2.2196329188725303
Validation Accuracy Epoch: 30.5
Training Loss per 5000 steps: 1.9785668849945068
Training Accuracy per 5000 steps: 43.75
The Total Accuracy for Epoch 1: 36.05714285714286
Training Loss Epoch: 2.0206801259898706
Training Accuracy Epoch: 36.05714285714286
Validation Loss Epoch: 2.0064546040126254
Validation Accuracy Epoch: 36.4
Training Loss per 5000 steps: 1.389538049697876
Training Accuracy per 5000 steps: 56.25
The Total Accuracy for Epoch 2: 49.8
Training Loss Epoch: 1.5858611467222101
Training Accuracy Epoch: 49.8
Validation Loss Epoch: 1.9639909948621477
Validation Accuracy Epoch: 40.15
Training Loss per 5000 steps: 0.8789005875587463
Training Accuracy per 5000 steps: 68.75
The Total Accuracy for Epoch 3: 63.885714285714286

In [46]:
acc = valid(model, test_dl)
print("Accuracy on test data = %0.2f%%" % acc)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Validation Loss Epoch: 2.660314120352268
Validation Accuracy Epoch: 41.3
Accuracy on test data = 41.30%
